In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

In [2]:
def Calculate_Alpha_Jensen(portfolio_returns: pd.Series,
                           benchmark_returns: pd.Series,
                           risk_free_rate: float = 0.2
                           ) -> float:
    # 2) Estimation de beta
    cov_pm = np.cov(portfolio_returns, benchmark_returns, ddof=0)[0, 0]
    var_m = np.var(benchmark_returns, ddof=0)
    beta = cov_pm / var_m

    # 3) Rendement moyen
    mean_port = np.mean(portfolio_returns)
    mean_bench = np.mean(benchmark_returns)

    # 4) Calcul de l'alpha
    expected_port = risk_free_rate + beta * (mean_bench - risk_free_rate)
    alpha = mean_port - expected_port

    return alpha

In [3]:
def Calculate_CAGR(portfolio_returns, annual_returns):
    # Paramètres
    V_fin = portfolio_returns.iloc[-1]
    V_debut = initiale_capital
    t = annual_returns

    # Calculer du CAGR
    CAGR = (((V_fin / V_debut) ** (1 / t)) - 1) * 100

    return CAGR

In [4]:
def Calculate_Volatility(returns, periods_per_year=12):
    vol = returns.std(ddof=1)
    vol *= np.sqrt(periods_per_year)

    return vol

In [5]:
def Calculate_Sharpe_Ratio(returns, vol, risk_free_rate=0.02, periods_per_year=12):
    returns = Calculate_CAGR(returns, len(returns))
    excess_returns = returns - (risk_free_rate / periods_per_year)
    mean_exc = excess_returns.mean() * periods_per_year

    return mean_exc / Calculate_Volatility(vol)

In [6]:
def Calculate_Total_Returns(portfolio_returns):
    Value_start = initiale_capital
    Value_final = portfolio_returns.iloc[-1]

    Total_Returns = (Value_final - Value_start) / Value_start

    return Total_Returns

In [7]:
def Calculate_Max_Drawdown(annual_returns):
    maxdrawdown = annual_returns.min()

    return maxdrawdown

In [8]:
daily_data = pd.DataFrame()
data_SP500 = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_SP500.parquet')
data_NDX = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet')
data_MP = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_MP.parquet')
data_Crypto = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_Crypto.parquet')
daily_data = pd.concat([data_SP500, data_NDX, data_MP, data_Crypto], ignore_index=True)
daily_data = daily_data.drop_duplicates(subset=['Ticker', 'Date'])

daily_data = daily_data.drop(columns=['Adj Close'])
# trier la data pour préparer la fusion
daily_data = daily_data.sort_values(by=['Date'], ascending=[True])

#Date au format Date
daily_data['Date'] = pd.to_datetime(daily_data['Date'])

# Définir la colonne 'Date' comme index
daily_data.set_index('Date', inplace=True)
daily_data

Price,Close,High,Low,Open,Volume,Ticker
Date,,,,,,
1976-07-01,0.409758,0.414986,0.409758,0.414986,1.274248e+06,BMY
1976-07-01,0.009944,0.010299,0.009855,0.009855,4.838400e+06,WMT
1976-07-01,0.292701,0.301708,0.292701,0.299456,3.264000e+05,TGT
1976-07-01,3.106392,3.106392,3.081932,3.081932,1.260000e+04,PNW
1976-07-01,0.236337,0.238884,0.235319,0.238375,2.980867e+06,F
...,...,...,...,...,...,...
2025-09-25,64.660004,64.930000,64.459999,64.800003,2.799300e+04,CL=F
2025-09-25,44.755001,44.764999,44.060001,44.119999,1.307500e+04,SI=F
2025-09-25,3781.000000,3785.600098,3760.500000,3768.300049,6.269400e+04,GC=F


In [9]:
weekly_data = daily_data.groupby("Ticker").resample('W').agg({
    'Open': 'first',   # Premier prix d'ouverture du mois
    'High': 'max',     # Plus haut du mois
    'Low': 'min',      # Plus bas du mois
    'Close': 'last',   # Dernier prix de clôture du mois
    'Volume': 'sum',   # Somme du volume sur le mois
}).reset_index()

In [10]:
# Calculer le rendement pour chaque Ticker avec un décalage d'un mois
weekly_data['Return'] = (weekly_data['Close'] / weekly_data['Open']) -1

# Paramètres
liste_ma = [9, 20, 25, 50, 100]

# Calcul des moyennes mobiles pour chaque fenêtre
for window in liste_ma:
    # Calcul de la moyenne mobile pour chaque 'Ticker'
    ma_column = f'ma_{window}'
    weekly_data[ma_column] = weekly_data.groupby('Ticker')['Close'].transform(lambda s: s.rolling(window, min_periods=1).mean())

liste_ma_1 = [9, 25, 50, 100]
liste_ma_2 = [9, 25, 50, 100]

for window in liste_ma_1:
    for window_2 in liste_ma_2:

        # Calcul de la moyenne mobile pour chaque 'Ticker'
        ma_column_1 = f'ma_{window}'
        ma_column_2 = f'ma_{window_2}'
        distance_ma_column = f'distance_ma_{window_2}/{window}'

        if ma_column_1 != ma_column_2:
            # Calcul du pourcentage d'écart entre le prix et la moyenne mobile
            weekly_data[distance_ma_column] = weekly_data[ma_column_2] / weekly_data[ma_column_1]


list_window = [4, 8, 12, 26, 52] # Liste de valeurs de top_n pour la sélection des meilleurs

# Calcul des performances pour chaque fenêtre
for window in list_window:

    # Nom dynamique de la colonne pour la performance
    roc_column = f'roc_{window}'

    # Calcul de la performance pour chaque 'Ticker' sur la fenêtre spécifiée
    weekly_data[roc_column] = weekly_data.groupby('Ticker')['Close'].pct_change(window)

liste_ma = [9, 25]
for window in liste_ma:

    # Calcul de la moyenne mobile pour chaque 'Ticker'
    distance_price_column = f'distance_price_{window}'
    ma_column = f'ma_{window}'

    # Calcul du pourcentage d'écart entre le prix et la moyenne mobile
    weekly_data[distance_price_column] = weekly_data['Close'] / weekly_data[ma_column]

weekly_data['Year'] = weekly_data['Date'].dt.year

# Close d’ancrage (première observation de l'année par Ticker)
anchor_close = weekly_data.groupby(['Ticker','Year'])['Close'].transform('first')

# Perf YTD (depuis le début d’année)
weekly_data['perf_ytd'] = weekly_data['Close'] / anchor_close - 1

list_window = [4, 8, 12, 26, 52] # Liste de valeurs de top_n pour la sélection des meilleurs

# Calcul des performances pour chaque fenêtre
for window in list_window:

    # Nom dynamique de la colonne pour la performance
    MAV_column = f'Momentum_Ajusted_Vol_{window}'
    Perf_column = f'Perf_{window}'
    Vol_column = f'Vol_{window}'

    # Calcul de la performance pour chaque 'Ticker' sur la fenêtre spécifiée
    weekly_data[Perf_column] = weekly_data.groupby('Ticker')['Close'].pct_change(window)
    weekly_data[Vol_column] = weekly_data.groupby('Ticker')['Return'].rolling(window=window).std().reset_index(level=0, drop=True)
    weekly_data[MAV_column] = weekly_data[Perf_column] / weekly_data[Vol_column]

period = 25

# Variation quotidienne
weekly_data['delta'] = weekly_data.groupby('Ticker')['Close'].diff()

# Gains et pertes
weekly_data['gains'] = weekly_data['delta'].clip(lower=0)
weekly_data['losses'] = -weekly_data['delta'].clip(upper=0)

# Moyenne mobile simple sur 'period' jours
weekly_data['avg_gain'] = (
    weekly_data.groupby('Ticker')['gains']
    .transform(lambda x: x.rolling(window=period, min_periods=period).mean())
)
weekly_data['avg_losses'] = (
    weekly_data.groupby('Ticker')['losses']
    .transform(lambda x: x.rolling(window=period, min_periods=period).mean())
)

# RS et RSI
weekly_data['rs'] = weekly_data['avg_gain'] / weekly_data['avg_losses']
weekly_data['rsi'] = 100 - (100 / (1 + weekly_data['rs']))

#weekly_data = weekly_data.drop(columns=['Return', 'ma_9', 'ma_25', 'ma_50', 'ma_100'])
weekly_data = weekly_data.dropna()

/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1967/3999834889.py:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  weekly_data[roc_column] = weekly_data.groupby('Ticker')['Close'].pct_change(window)
/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1967/3999834889.py:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  weekly_data[roc_column] = weekly_data.groupby('Ticker')['Close'].pct_change(window)
/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1967/3999834889.py:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is 

In [11]:
# Initialiser la colonne Prediction à 0
weekly_data["Prediction"] = 0

# Identifier les indices du top 30 par date
top30_idx = (
    weekly_data
    .sort_values(by=["Date", "perf_ytd"], ascending=[True, False])
    .groupby("Date")
    .head(10)
    .index
)

# Mettre Prediction = 1 pour ces lignes
weekly_data.loc[top30_idx, "Prediction"] = 1

In [12]:
daily_data = daily_data.reset_index()

In [13]:
daily_data['Return'] = daily_data.groupby('Ticker')['Close'].pct_change()
daily_data = daily_data[['Date', 'Close', 'Open', 'Ticker', 'Return']]
weekly_data = weekly_data.drop(columns=['Close', 'Open', 'Return'])
weekly_data = weekly_data.sort_values(by=['Date'], ascending=[True])

In [14]:

start_date = "2000-01-01"
daily_data = daily_data.reset_index()

daily_data = daily_data[daily_data['Date'] >= start_date]
weekly_data = weekly_data[weekly_data['Date'] >= start_date]

weekly_data = weekly_data.replace([np.inf, -np.inf], 0)
#weekly_data = weekly_data.drop(columns=['Close', 'Open', 'High', 'Low', 'Volume'])
# Remplacer les valeurs NaN par 0
weekly_data = weekly_data.dropna()


In [15]:
data = pd.merge_asof(daily_data, weekly_data, on='Date', by='Ticker', direction='backward')
data['weekday'] = data['Date'].dt.dayofweek
data = data[data['weekday'] < 5]

In [16]:
def backtest(
    data: pd.DataFrame,
    initial_capital: float = 1000,
    save_month: float = 0,
    fee_per_trade: float = 0,
    moving_average_buy: int = 9,
    moving_average_sell: int = 25,
    proba_column: str = "Proba",
    max_positions: int = 10,
    leverage: float = 1.0,
    weight: float = 1.0,
):
    """
    Backtest vectorisé avec pandas :
      - data : DataFrame (Date, Ticker, Open, Close, Prediction, Return, Crisis, ma_xx, Proba, ...)
      - stop_loss : seuil de perte à partir du prix d’entrée (optionnel)
      - retourne aussi la liste des tickers détenus par jour
    """

    ma_buy  = f"ma_{moving_average_buy}"
    ma_sell = f"ma_{moving_average_sell}"

    required = ["Date","Ticker","Open","Close","Prediction","Return",ma_buy,ma_sell, 'rsi']
    missing = [c for c in required if c not in data.columns]
    if (missing or max_positions >weight):
        raise ValueError(f"Colonnes manquantes: {missing}")

    df = data.sort_values(["Date", "Ticker"]).copy()

    # --- 1. Préparer signaux (vectorisé) ---
    df["BuySignal"] = (

        (df["Prediction"] == 1)
    )

    df["SellSignal"] = (
        (df["Prediction"] == 0
    ))

    # Poids (score de tri pour sélectionner max_positions)
    if proba_column in df.columns:
        df["Score"] = df[proba_column].fillna(0.0)
    else:
        df["Score"] = (df["Close"] / df[ma_buy] - 1.0).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Décalage Return pour obtenir rendement t+1 directement
    df["NextReturn"] = df.groupby("Ticker")["Return"].shift(-1)

    # --- 2. Boucle sur dates (plus légère) ---
    unique_dates = df["Date"].drop_duplicates().sort_values().to_numpy()
    capital = initial_capital

    portfolio_values, returns, n_positions, fees_hist, mean_return_history = [], [], [], [], []
    tickers_history = {}  # 🔥 tickers détenus par jour

    prev_holdings = set()

    for date in unique_dates:
        day = df.loc[df["Date"] == date]

        # Ajouter capital mensuel
        capital += save_month

        # Gestion des signaux
        buy_set  = set(day.loc[day["BuySignal"], "Ticker"])
        sell_set = set(day.loc[day["SellSignal"], "Ticker"])

        # Portefeuille courant
        held_after_sell = prev_holdings - sell_set
        new_buys = buy_set - prev_holdings

        # Tri par Score
        scores = day.set_index("Ticker")["Score"].to_dict()
        held_sorted = sorted(list(held_after_sell), key=lambda t: scores.get(t, 0.0), reverse=True)

        if len(held_sorted) >= max_positions:
            current_portfolio = set(held_sorted[:max_positions])
        else:
            remaining = max_positions - len(held_sorted)
            new_sorted = sorted(list(new_buys), key=lambda t: scores.get(t, 0.0), reverse=True)
            add_these = new_sorted[:remaining]
            current_portfolio = set(held_sorted + add_these)

        # Frais
        opened = current_portfolio - prev_holdings
        closed = prev_holdings - current_portfolio
        trades = len(opened) + len(closed)
        fees = trades * fee_per_trade
        capital -= fees

        # Calcul rendement t+1
        r = day.loc[day["Ticker"].isin(current_portfolio), "NextReturn"].dropna().to_numpy()
        l = day.loc[day["Ticker"].isin(current_portfolio), ("NextReturn", 'Ticker')]
        if r.size > 0:
            mean_return = np.mean(r) * leverage
            invested_fraction = len(r) / weight  # part du capital investi
            capital *= 1 + invested_fraction * mean_return
        else:
            mean_return = 0.0
            capital *= (1 + mean_return)

        # Historique
        portfolio_values.append(capital)
        returns.append(mean_return)
        n_positions.append(len(current_portfolio))
        fees_hist.append(fees)
        tickers_history[date] = l # 🔥 tickers détenus
        mean_return_history.append(mean_return)

        prev_holdings = current_portfolio

    # --- Résultats ---
    portfolio_per_day = pd.DataFrame({
        "Date": unique_dates,
        "Capital": portfolio_values,
        "Return": np.array(returns) * 100.0,
        "Ticker_Count": n_positions,
        "Fees": fees_hist,
        "Return_Mean": mean_return_history,
    }).set_index("Date")

    monthly = portfolio_per_day.resample("ME").agg({
        "Capital": "last",
        "Ticker_Count": "mean",
        "Fees": "sum",
        "Return_Mean": "mean",
    })
    first_cap_m = portfolio_per_day["Capital"].resample("ME").first().ffill()
    last_cap_m  = portfolio_per_day["Capital"].resample("ME").last().ffill()
    monthly["Return"] = ((last_cap_m / first_cap_m - 1.0) * 100.0).fillna(0.0)

    annual = portfolio_per_day.resample("YE").agg({
        "Capital": "last",
        "Ticker_Count": "mean",
        "Fees": "sum",
        "Return_Mean": "mean"
    })
    first_cap_y = portfolio_per_day["Capital"].resample("YE").first().ffill()
    last_cap_y  = portfolio_per_day["Capital"].resample("YE").last().ffill()
    annual["Return"] = ((last_cap_y / first_cap_y - 1.0) * 100.0).fillna(0.0)

    return portfolio_per_day, monthly, annual, tickers_history

In [17]:
initiale_capital = 1000
portfolio_per_day, portfolio_per_month, portfolio_per_annual, list_ticker = backtest(data, initial_capital=initiale_capital, save_month=0, fee_per_trade=1, moving_average_buy=9, moving_average_sell=25, max_positions=10, leverage=1, weight=10)

In [18]:
worst = portfolio_per_month[portfolio_per_month['Return'] < 0]
worst = worst.sort_values(by=['Return'], ascending=True)
print(len(portfolio_per_month), len(worst), len(worst)/len(portfolio_per_month)*100)
worst.head(20)

309 119 38.51132686084142


,Capital,Ticker_Count,Fees,Return_Mean,Return
Date,,,,,
2000-03-31,847.622387,10.0,12,-0.015667,-30.839112
2003-01-31,321.054485,10.0,64,-0.008358,-29.210619
2020-03-31,6720.703904,10.0,36,-0.012057,-23.636216
2022-06-30,38514.151094,10.0,12,-0.010917,-21.585548
2001-09-30,579.859900,10.0,8,-0.012913,-20.531334
2008-11-30,953.046690,10.0,18,-0.009415,-19.781465
2012-05-31,1720.803140,10.0,12,-0.006937,-16.449498
2009-02-28,766.481367,10.0,34,-0.007394,-16.392863
2018-10-31,7682.683086,10.0,10,-0.008113,-15.521869


In [19]:
worst_mean_return = portfolio_per_month[portfolio_per_month['Return_Mean'] < 0][['Ticker_Count', 'Fees', 'Return_Mean']]
worst_mean_return = worst_mean_return.sort_values(by=['Return_Mean'], ascending=True)
print(len(portfolio_per_month), len(worst_mean_return), len(worst_mean_return)/len(portfolio_per_month)*100)
worst_mean_return.describe()

309 102 33.00970873786408


,Ticker_Count,Fees,Return_Mean
count,102.000000,102.000000,102.000000
mean,9.999533,17.745098,-0.003323
std,0.004715,13.706027,0.003168
min,9.952381,2.000000,-0.015667
25%,10.000000,10.000000,-0.004954
50%,10.000000,12.000000,-0.002588
75%,10.000000,20.000000,-0.000751
max,10.000000,64.000000,-0.000024


In [20]:
best_mean_return = portfolio_per_month[portfolio_per_month['Return_Mean'] > 0]
best_mean_return = best_mean_return.sort_values(by=['Return_Mean'], ascending=False)
print(len(portfolio_per_month), len(best_mean_return), len(best_mean_return)/len(portfolio_per_month)*100)
best_mean_return.describe()

309 207 66.99029126213593


,Capital,Ticker_Count,Fees,Return_Mean,Return
count,207.000000,207.000000,207.000000,207.000000,207.000000
mean,14990.903812,9.998409,17.333333,0.003890,6.933638
std,30349.044675,0.009739,15.650305,0.003749,8.777205
min,298.646625,9.904762,2.000000,0.000002,-7.012615
25%,1185.786227,10.000000,8.000000,0.001302,1.455770
50%,2155.262722,10.000000,12.000000,0.002783,4.948397
75%,8492.430124,10.000000,20.000000,0.005477,9.166261
max,157802.932398,10.000000,76.000000,0.023587,57.457206


In [21]:
portfolio_per_day['Ticker_Count'].describe()

count    6583.000000
mean        9.998785
std         0.038959
min         8.000000
25%        10.000000
50%        10.000000
75%        10.000000
max        10.000000
Name: Ticker_Count, dtype: float64

In [22]:
portfolio_per_month

,Capital,Ticker_Count,Fees,Return_Mean,Return
Date,,,,,
2000-01-31,885.081183,10.0,54,-0.002924,-7.833105
2000-02-29,1270.906341,10.0,30,0.020209,42.418329
2000-03-31,847.622387,10.0,12,-0.015667,-30.839112
2000-04-30,828.213011,10.0,34,0.002474,1.460107
2000-05-31,803.396497,10.0,20,0.000220,2.795960
...,...,...,...,...,...
2025-05-31,124912.726641,10.0,20,0.002152,3.988304
2025-06-30,125646.098604,10.0,16,0.000320,0.214148
2025-07-31,130811.649667,10.0,20,0.001811,3.856816


In [23]:
portfolio_per_annual

,Capital,Ticker_Count,Fees,Return_Mean,Return
Date,,,,,
2000-12-31,932.979213,10.000000,214,0.001241,-2.845299
2001-12-31,752.805721,10.000000,210,0.000526,-18.375791
2002-12-31,452.758226,10.000000,222,-0.000552,-38.585004
2003-12-31,842.064270,10.000000,170,0.004458,85.666956
2004-12-31,999.281869,10.000000,214,0.001870,14.221171
2005-12-31,1456.703269,10.000000,208,0.002417,51.297996
2006-12-31,1559.716535,10.000000,216,0.001129,7.493460
2007-12-31,2155.389831,10.000000,216,0.001971,38.191125
2008-12-31,1061.209299,10.000000,256,-0.001817,-50.743365


In [24]:
list_ticker

{np.datetime64('2000-01-03T00:00:00.000000000'): Price  NextReturn Ticker
 254     -0.084310   AAPL
 113     -0.028572    ABT
 165     -0.005435   ACGL
 0       -0.083889   ADBE
 210     -0.050589    ADI
 49      -0.010417    ADM
 97       0.000000    ADP
 95      -0.048689   ADSK
 142      0.000000    AEE
 22       0.011928    AEP,
 np.datetime64('2000-01-04T00:00:00.000000000'): Price  NextReturn Ticker
 508      0.014633   AAPL
 430     -0.001838    ABT
 322      0.092896   ACGL
 598      0.019771   ADBE
 346      0.014599    ADI
 526     -0.015789    ADM
 407     -0.009616    ADP
 447     -0.062992   ADSK
 606      0.038685    AEE
 610      0.037328    AEP,
 np.datetime64('2000-01-05T00:00:00.000000000'): Price  NextReturn Ticker
 866     -0.086538   AAPL
 928      0.034991    ABT
 726     -0.010000   ACGL
 773      0.008163   ADBE
 634     -0.027339    ADI
 827      0.005347    ADM
 929      0.013350    ADP
 886     -0.060925   ADSK
 679     -0.003725    AEE
 685      0.005682    

In [25]:
list_ticker.items()

dict_items([(np.datetime64('2000-01-03T00:00:00.000000000'), Price  NextReturn Ticker
254     -0.084310   AAPL
113     -0.028572    ABT
165     -0.005435   ACGL
0       -0.083889   ADBE
210     -0.050589    ADI
49      -0.010417    ADM
97       0.000000    ADP
95      -0.048689   ADSK
142      0.000000    AEE
22       0.011928    AEP), (np.datetime64('2000-01-04T00:00:00.000000000'), Price  NextReturn Ticker
508      0.014633   AAPL
430     -0.001838    ABT
322      0.092896   ACGL
598      0.019771   ADBE
346      0.014599    ADI
526     -0.015789    ADM
407     -0.009616    ADP
447     -0.062992   ADSK
606      0.038685    AEE
610      0.037328    AEP), (np.datetime64('2000-01-05T00:00:00.000000000'), Price  NextReturn Ticker
866     -0.086538   AAPL
928      0.034991    ABT
726     -0.010000   ACGL
773      0.008163   ADBE
634     -0.027339    ADI
827      0.005347    ADM
929      0.013350    ADP
886     -0.060925   ADSK
679     -0.003725    AEE
685      0.005682    AEP), (np.dateti

In [26]:
# 1) Calculer le drawdown mensuel
#    drawdown = (Capital / pic_historique) - 1
portfolio_per_day['Peak'] = portfolio_per_day['Capital'].cummax()
portfolio_per_day['Drawdown'] = portfolio_per_day['Capital'] / portfolio_per_day['Peak'] - 1

# 2) Résultat : série mensuelle de drawdowns
#    Le drawdown est négatif (0 au pic, baisse ensuite)  [oai_citation:0‡reddit.com](https://www.reddit.com/r/learnpython/comments/bxyze5/getting_max_drawdown_with_python/?utm_source=chatgpt.com)

# 3) Calculer le max drawdown global
max_dd_global = portfolio_per_day['Drawdown'].min()

# 4) Calculer le drawdown maximal par année
#    On regroupe par année et on prend le minimum (le plus négatif) de Drawdown
mensuel_max_dd = (
    portfolio_per_day['Drawdown']
    .groupby(portfolio_per_day.index.year)
    .min()
    .rename('MaxDrawdown')
    .to_frame()
)

# Calcul du nombre d'actions sélectionnées chaque mois
monthly_counts = data.groupby('Date')['Prediction'].sum()
monthly_counts.index = pd.to_datetime(monthly_counts.index)

# Moyenne des sélections mensuelles
average_count = monthly_counts.mean()

# 1) Regrouper par année (resample('Y') pour Year-End)
annual_counts = monthly_counts.resample('YE').sum()

# 2) Calculer la moyenne annuelle des sélections
average_annual_count = annual_counts.mean()

In [27]:
data = data.sort_values('Date')
# 1) Télécharger les cours mensuels ajustés du S&P 500
ndx = yf.download(
    "^GSPC",
    start=data['Date'].iloc[0],
    end=data['Date'].iloc[-1],
    interval="1d",
    auto_adjust=True,
    actions=False
)["Close"]
# 2) Convertir l’index en période mensuelle, puis au TIMESTAMP de fin de mois
ndx.index = pd.to_datetime(ndx.index)

# 3) Normaliser pour que le S&P 500 commence à la même valeur que le capital initial
ndx_return = (ndx.pct_change())

# 4) Reindexer ndx_norm sur l’index de portfolio_per_month (remplit les mois manquants par propagation)
ndx_return = ndx_return.reindex(portfolio_per_day.index, method="ffill")

# 5) Fusionner dans df_bench
portfolio_per_day["S&P500_return"] = ndx_return
portfolio_per_day["S&P500_return"].fillna(0)

initial_cap_ndx = portfolio_per_day["Capital"].iloc[0]
portfolio_per_day["S&P500_capital"] = (1 + portfolio_per_day["S&P500_return"]).cumprod() * initial_cap_ndx
portfolio_per_day['S&P500_return'] = portfolio_per_day['S&P500_return'] * 100

# 1) Calculer le drawdown mensuel
#    drawdown = (Capital / pic_historique) - 1
portfolio_per_day['S&P500_Peak'] = portfolio_per_day['S&P500_capital'].cummax()
portfolio_per_day['S&P500_Drawdown'] = portfolio_per_day['S&P500_capital'] / portfolio_per_day['S&P500_Peak'] - 1

# 3) Calculer le max drawdown global
max_dd_global_ndx = portfolio_per_day['S&P500_Drawdown'].min()

# 4) Calculer le drawdown maximal par année
#    On regroupe par année et on prend le minimum (le plus négatif) de Drawdown
mensuel_max_dd_ndx = (
    portfolio_per_day['S&P500_Drawdown']
    .groupby(portfolio_per_day.index.year)
    .min()
    .rename('MaxDrawdown')
    .to_frame()
)

portfolio_per_month = portfolio_per_day.resample('ME').agg({
    'Capital': 'last',
    'Ticker_Count': 'mean',
    'Drawdown': 'min',
    'S&P500_capital': 'last',
    'S&P500_Drawdown': 'min'
})
first_capital_month = portfolio_per_day['Capital'].resample('ME').first().ffill()
last_capital_month = portfolio_per_day['Capital'].resample('ME').last().ffill()
first_capital_ndx_month = portfolio_per_day['S&P500_capital'].resample('ME').first().ffill()
last_capital_ndx_month = portfolio_per_day['S&P500_capital'].resample('ME').last().ffill()

portfolio_per_month['Return'] = ((last_capital_month / first_capital_month - 1) * 100).fillna(0)
portfolio_per_month['S&P500_return'] = ((last_capital_ndx_month / first_capital_ndx_month - 1) * 100).fillna(0)

portfolio_per_annual = portfolio_per_day.resample('YE').agg({
    'Capital': 'last',
    'Ticker_Count': 'mean',
    'Drawdown': 'min',
    'S&P500_capital': 'last',
    'S&P500_Drawdown': 'min'
})
first_capital_year = portfolio_per_day['Capital'].resample('YE').first().ffill()
first_capital_ndx_year = portfolio_per_day['S&P500_capital'].resample('YE').first().ffill()
last_capital_year = portfolio_per_day['Capital'].resample('YE').last().ffill()
last_capital_ndx_year = portfolio_per_day['S&P500_capital'].resample('YE').first().ffill()

portfolio_per_annual['Return'] = ((last_capital_year / first_capital_year - 1) * 100).fillna(0)
portfolio_per_annual['S&P500_return'] = ((last_capital_ndx_year / first_capital_ndx_year - 1) * 100).fillna(0)

[*********************100%***********************]  1 of 1 completed


## Metrics

In [28]:
CAGR_Portfolio = Calculate_CAGR(portfolio_per_annual['Capital'], len(portfolio_per_annual)).round(2)
CAGR_ndx = Calculate_CAGR(portfolio_per_annual['S&P500_capital'], len(portfolio_per_annual)).round(2)
print(f'CAGR_Portfolio:{CAGR_Portfolio}')
print(f'CAGR_ndx:{CAGR_ndx}')

CAGR_Portfolio:21.1
CAGR_ndx:6.33


In [29]:
Alpha_Portfolio = Calculate_Alpha_Jensen(portfolio_per_day['Return'], portfolio_per_day['S&P500_return'])
print(f'Alpha_Portfolio:{Alpha_Portfolio}')


Alpha_Portfolio:0.44757232348012216


In [30]:
Sharpe_Portfolio = Calculate_Sharpe_Ratio(portfolio_per_month['Capital'], portfolio_per_month['Return']).round(2)
Sharpe_ndx = Calculate_Sharpe_Ratio(portfolio_per_month['S&P500_capital'], portfolio_per_month['S&P500_return']).round(2)
print(f'Sharpe_Portfolio:{Sharpe_Portfolio}')
print(f'Sharpe_ndx:{Sharpe_ndx}')

Sharpe_Portfolio:0.54
Sharpe_ndx:0.39


In [31]:
Total_Returns_Portfolio = Calculate_Total_Returns(portfolio_per_month['Capital']).round(2)
Total_Returns_ndx = Calculate_Total_Returns(portfolio_per_month['S&P500_capital']).round(2)
print(f'Total_Return_Portfolio:{Total_Returns_Portfolio}')
print(f'Total_Return_ndx:{Total_Returns_ndx}')

Total_Return_Portfolio:144.23
Total_Return_ndx:3.94


In [32]:
Max_Drawdown_Portfolio = Calculate_Max_Drawdown(portfolio_per_day['Return'])
Max_Drawdown_ndx = Calculate_Max_Drawdown(portfolio_per_day['S&P500_return'])
print(f'Max_Drawdown_Portfolio:{Max_Drawdown_Portfolio}')
print(f'Max_Drawdown_ndx:{Max_Drawdown_ndx}')

Max_Drawdown_Portfolio:-17.52242453379728
Max_Drawdown_ndx:-11.984055240393443


In [33]:
Volatility_Portfolio = Calculate_Volatility(portfolio_per_month['Return']).round(2)
Volatility_ndx = Calculate_Volatility(portfolio_per_month['S&P500_return']).round(2)
print(f'Volatility_Portfolio:{Volatility_Portfolio}')
print(f'Volatility_ndx:{Volatility_ndx}')

Volatility_Portfolio:36.31
Volatility_ndx:15.86


In [34]:
# 6) Tracer l’évolution du capital vs benchmark (échelle log)
fig = px.line(
    portfolio_per_month.reset_index(),
    x="Date",
    y=["Capital", "S&P500_capital"],
    title="Évolution du capital vs S&P 500 (benchmark)",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)

fig.update_yaxes(type="log")  # axe Y en échelle logarithmique
fig.update_layout(
    xaxis_tickformat="%Y-%m",
    xaxis_title="Mois",
    yaxis_title="Valeur (log)",
    legend_title="Courbes"
)
fig.show()

fig = px.bar(
    portfolio_per_month.reset_index(),
    x="Date",
    y=["Return", "S&P500_return"],
    barmode="group",
    title="Return evolution vs S&P500 (benchmark)",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)
# axe Y en échelle logarithmique
fig.update_layout(
    xaxis_tickformat="%Y-%m",
    xaxis_title="Mois",
    yaxis_title="Valeur (log)",
    legend_title="Courbes"
)
fig.show()

fig = px.area(
    portfolio_per_month.reset_index(),
    x='Date',
    y=["Drawdown", "S&P500_Drawdown"],
    title="Drawdown mensuel",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)
fig.update_layout(
    xaxis_tickformat='%Y-%m',
    yaxis_tickformat='%.0%',
    yaxis_title="Drawdown",
)
fig.show()

fig = px.bar(
    portfolio_per_month.reset_index(),
    x="Date",
    y="Ticker_Count",
    title="Counts tickers by month",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)
# axe Y en échelle logarithmique
fig.update_layout(
    xaxis_tickformat="%Y-%m",
    xaxis_title="Mois",
    yaxis_title="Valeur (log)",
    legend_title="Courbes"
)
fig.show()

fig = px.line(
    portfolio_per_annual.reset_index(),
    x="Date",
    y=["Capital", "S&P500_capital"],
    title="Évolution du capital vs S&P500 (benchmark)",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)

fig.update_yaxes(type="log")  # axe Y en échelle logarithmique
fig.update_layout(
    xaxis_tickformat="%Y-%m",
    xaxis_title="Mois",
    yaxis_title="Valeur (log)",
    legend_title="Courbes"
)
fig.show()

fig = px.bar(
    portfolio_per_annual.reset_index(),
    x="Date",
    y=["Return", "S&P500_return"],
    barmode="group",
    title="Return evolution vs S&P500 (benchmark)",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)
# axe Y en échelle logarithmique
fig.update_layout(
    xaxis_tickformat="%Y-%m",
    xaxis_title="Mois",
    yaxis_title="Valeur (log)",
    legend_title="Courbes"
)
fig.show()

fig = px.area(
    portfolio_per_annual.reset_index(),
    x='Date',
    y=["Drawdown", "S&P500_Drawdown"],
    title="Drawdown annuel",
    labels={"value": "Valeur", "variable": "Série", "Date": "Mois"}
)
fig.update_layout(
    xaxis_tickformat='%Y-%m',
    yaxis_tickformat='%.0%',
    yaxis_title="Drawdown",
)
fig.show()